In [55]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import pad

import pandas as pd
import numpy as np



In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [182]:
# Hyperparams
input_size = 1
sequence_length = 1500
num_layers = 2
hidden_size = 1500
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [217]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # x -> batch_size, sequence_length, input_size


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        # out: batch_size, seq_length, hidden_size
        out = out[:, -1, :]
        # out = out.reshape((1, 1, 1500)) # this stops warning
        return out
        

In [222]:
model = RNN(input_size, hidden_size, num_layers).double()

In [223]:
class SequenceDataset(Dataset):
    def __init__(self, transform):
        self.data = pd.read_pickle('../data/rnn/data.csv')
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # y = output[index].reshape((1, -1)) # make the data match the shape of X after passing through all layers
        x = self.data['in'].iloc[index].reshape(1, -1)
        x = np.array(x, dtype=np.float32)
        
        # x = x.T
        y = self.data['out'].iloc[index]
        y = y.reshape((1,-1))
        print(y.shape)

        if self.transform:
            x = self.transform(x)[0]
            # print(x.dtype)

        return x, y

In [224]:
dataset = SequenceDataset(transforms.ToTensor())
length = len(pd.read_pickle('../data/cnn/data.csv'))
train_len = (length * 9) // 10
test_len = length - train_len
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])

train_loader = DataLoader(dataset=train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=True)

In [225]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (data, target) in enumerate(train_loader):
        data = data.reshape(-1, sequence_length, input_size).to(device)
        target = target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch: [{epoch+1}/{num_epochs}], step: [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')

(1, 1500)


RuntimeError: mat1 and mat2 must have the same dtype